In [ ]:
#  Copyright 2023 Google LLC
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      https://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.

# Controlled state preparation using rotations

In [ ]:
from qualtran.bloqs.state_preparation.state_preparation_via_rotation import StatePreparationViaRotations
from qualtran.drawing import show_bloq
from qualtran import BloqBuilder
from qualtran.bloqs.basic_gates import ZeroState, OneState, OneEffect, PlusState, CNOT
from qualtran.bloqs.rotations.phase_gradient import PhaseGradientState
import numpy as np
import random

This bloq prepares a state $|\psi\rangle$ given a list of its coefficients controlled by another qubit. It uses phase kickback on a gradient state register to perform the rotations, so such state must be provided. It can be obtained from `PhaseGradientState`.

Refer to https://arxiv.org/abs/1812.00954 page 3 for more details on state preparation using rotations.

## Example of use

Assume one wants to implement an arbitrary state whose coefficients are stored in `state_coefs` with `state_bitsizes` sites using a resolution for the rotations of `phase_bitsizes` qubits.

In [ ]:
def gen_random_state (state_bitsizes: int):
    state = np.array([random.uniform(-1,1) + random.uniform(-1,1)*1j for _ in range(2**state_bitsizes)])
    return state/np.linalg.norm(state)


random.seed(137)
phase_bitsize = 4
state_bitsize = 3
state_coefs = gen_random_state(state_bitsize)

## Building the bloq

The parameters that the bloq receives are:

  - `phase_bitsize`: number of qubits used to store the rotation angle. This determines the accuracy of the results, but increases computational resources.
  - `state_coefficients`: tuple that contains the coefficients of the quantum state to be encoded, must be of length a power of two.
  - `uncompute`: boolean flag to implement the adjoint of the gate.
  - `control_bitsize`: number of qubits of the control register. Set to zero (default value) for an uncontrolled gate.

Below the bloq and its decomposition are shown. It is possible to see three big bloqs that do the sequential rotations to prepare the amplitude for each of the three qubits and a final bloq to encode the phases of the state.

In [ ]:
qsp = StatePreparationViaRotations(
    phase_bitsize=phase_bitsize, state_coefficients=tuple(state_coefs)
)
show_bloq(qsp)
show_bloq(qsp.decompose_bloq())

## Using the bloq in a circuit

Now let us show an example of this bloq being used to encode a state, together with the tensor contract to ensure that the coefficients are correctly prepared.

In [ ]:
bb = BloqBuilder()
state = bb.allocate(state_bitsize)
phase_gradient = bb.add(PhaseGradientState(phase_bitsize))
state, phase_gradient = bb.add(
    qsp, target_state=state, phase_gradient=phase_gradient
)
bb.add(PhaseGradientState(bitsize=phase_bitsize).adjoint(), phase_grad=phase_gradient)
circuit = bb.finalize(state=state)

show_bloq(circuit)
coefficients = circuit.tensor_contract()

And finally a comparison of the results obtained with the original state used.

In [ ]:
accuracy = np.dot(coefficients, np.array(state_coefs).conj())

print(f"original state used: {tuple(state_coefs)}")
print(f"circuit result:      {tuple(coefficients)}\n")
print(f"accuracy: {abs(accuracy)}\n")

print("Comparison (coefficients in polar form):")
for i, (c, s) in enumerate(zip(coefficients, state_coefs)):
    print(f"  |{i:0{state_bitsize}b}> result: {round(abs(c),4)} ∠{round(np.angle(c, deg=True),2)}º  "+\
          f"exact: {round(abs(s),4)} ∠{round(np.angle(s, deg=True),2)}º")

## Controlled state preparation

Below is an example of the same state preparation gate but in this case with a two qubit control register that is in the $|+\rangle$ state. Thus, the result of applying the gate $U$, which prepares the state $|\psi\rangle$ is

$$
\frac{1}{\sqrt{2}}U(|0,0\rangle + |1,0\rangle) = \frac{1}{\sqrt{2}}(|0,0\rangle + |1,\psi\rangle)
$$

In [ ]:
qsp_ctrl = StatePreparationViaRotations(
    phase_bitsize=phase_bitsize,
    state_coefficients=tuple(state_coefs),
    control_bitsize=1
)
bb = BloqBuilder()
control = bb.add(PlusState())
state = bb.allocate(state_bitsize)
phase_gradient = bb.add(PhaseGradientState(phase_bitsize))
control, state, phase_gradient = bb.add(
    qsp_ctrl, prepare_control=control, target_state=state, phase_gradient=phase_gradient
)
bb.add(PhaseGradientState(bitsize=phase_bitsize).adjoint(), phase_grad=phase_gradient)
network = bb.finalize(control=control, state=state)
coefficients = network.tensor_contract()
correct = 1 / np.sqrt(2) * np.array([1] + [0] * (2**state_bitsize - 1) + list(state_coefs))

And again, a comparison with the desired result

In [ ]:
accuracy = np.dot(coefficients, np.array(correct).conj())

print(f"accuracy: {abs(accuracy)}\n")

print("Comparison (coefficients in polar form):")
for i, (c, s) in enumerate(zip(coefficients, correct)):
    print(f"  |{f'{i:0{state_bitsize+1}b}'[0]},{f'{i:0{state_bitsize+1}b}'[1:]}> result: {round(abs(c),4)} ∠{round(np.angle(c, deg=True),2)}º  "+\
          f"exact: {round(abs(s),4)} ∠{round(np.angle(s, deg=True),2)}º")

## Using the adjoint

This block also implement the adjoint through the parameter `uncompute`, that is, preparing $|0\rangle$ from a given $|\psi\rangle$. Following an equivalent scheme to the previous example, we prepare and un-prepare a state using the adjoint $U^\dagger$ of the preparation gate $U$:
$$
  |0\rangle = U^\dagger |\psi\rangle = U^\dagger U |0\rangle
$$

In [ ]:
qsp_adj = StatePreparationViaRotations(
    phase_bitsize=phase_bitsize, state_coefficients=tuple(state_coefs), uncompute=True
)

bb = BloqBuilder()
state = bb.allocate(state_bitsize)
phase_gradient = bb.add(PhaseGradientState(phase_bitsize))
state, phase_gradient = bb.add(
    qsp, target_state=state, phase_gradient=phase_gradient
)
state, phase_gradient = bb.add(
    qsp_adj, target_state=state, phase_gradient=phase_gradient
)
bb.add(PhaseGradientState(bitsize=phase_bitsize).adjoint(), phase_grad=phase_gradient)
circuit = bb.finalize(state=state)

show_bloq(circuit)
coefficients = circuit.tensor_contract()

accuracy = coefficients[0] # <coefficients|0> = coefficients[0]
print(f"accuracy: {abs(accuracy)}\n")

print("Coefficients in polar form:")
# zero out small coefficients
coefficients[np.where(abs(coefficients) < 1e-16)] = 0.0
for i, c in enumerate(coefficients):
    print(f"  |{i:0{state_bitsize}b}> result: {round(abs(c),4)} ∠{round(np.angle(c, deg=True),2)}º")